In [ ]:
# scripts/inference.py

import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBClassifier

def load_data(test_path):
    """
    Загрузка тестовых данных.
    """
    test = pd.read_csv(test_path)
    return test

def preprocess_data(data, scaler, encoder):
    """
    Предобработка данных:
    - Заполнение пропущенных значений медианой для числовых признаков
    - One-Hot Encoding для категориальных признаков
    - Масштабирование числовых признаков
    - Удаление ненужных признаков (например, 'ID')
    """
    # Сохранение 'ID' для финального файла
    test_ids = data['ID']

    # Удаление 'ID' из данных перед предобработкой
    data = data.drop(['ID'], axis=1)

    # Определение числовых и категориальных признаков
    numerical_features = ['age', 'resting_blood_pressure', 'serum_cholestoral',
                          'maximum_heart_rate_achieved', 'oldpeak', 'number_of_major_vessels']
    categorical_features = ['sex']

    # Заполнение пропущенных значений медианой для числовых признаков
    for feature in numerical_features:
        if data[feature].isnull().sum() > 0:
            median = data[feature].median()
            data[feature].fillna(median, inplace=True)

    # One-Hot Encoding для категориальных признаков
    encoded_categorical = encoder.transform(data[categorical_features])
    encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_features))

    # Объединение закодированных категориальных признаков с исходными данными
    data = pd.concat([data.reset_index(drop=True), encoded_categorical_df.reset_index(drop=True)], axis=1)

    # Удаление исходных категориальных признаков
    data.drop(categorical_features, axis=1, inplace=True)

    # Масштабирование числовых признаков
    data[numerical_features] = scaler.transform(data[numerical_features])

    return data, test_ids

def predict(model, data):
    """
    Выполнение предсказаний с использованием модели XGBoost.
    """
    predictions = model.predict(data)
    probabilities = model.predict_proba(data)[:,1]
    return predictions, probabilities

def main():
    # Пути к файлам
    test_path = 'test.csv'  # Убедитесь, что путь корректен
    output_path = 'inference_submission.csv'

    # Загрузка тестовых данных
    test = load_data(test_path)

    # Проверка наличия столбца 'ID'
    if 'ID' not in test.columns:
        raise ValueError("В тестовых данных отсутствует столбец 'ID'.")

    # Загрузка масштабатора и энкодера
    scaler = joblib.load('scaler.joblib')
    encoder = joblib.load('encoder.joblib')

    # Предобработка данных
    test_processed, test_ids = preprocess_data(test, scaler, encoder)

    # Загрузка модели XGBoost
    model = joblib.load('xgboost_model.joblib')

    # Выполнение предсказаний
    predictions, probabilities = predict(model, test_processed)

    # Создание DataFrame для сохранения результатов
    submission = pd.DataFrame({
        'ID': test_ids,
        'class': predictions
    })

    # Сохранение результатов
    submission.to_csv(output_path, index=False)
    print(f"Инференс завершён. Результаты сохранены в '{output_path}'.")

if __name__ == "__main__":
    main()
